# 🚀 TRANSMUTATION PRODUCTION — Batch Processing

```
╔══════════════════════════════════════════════════════════════════════════════╗
║                 FRÉGATE 01_TRANSMUTATION — PRODUCTION MODE                   ║
║              Batch Processing • Multi-Actor • Full Pipeline                  ║
╚══════════════════════════════════════════════════════════════════════════════╝
```

**Mission**: Traiter plusieurs acteurs en batch via le pipeline TRANSMUTATION complet.

---

## 📦 Setup Production Environment

In [ ]:
# Mount Drive
from google.colab import drive
drive.mount('/content/drive')

# Configuration
DRIVE_ROOT = "/content/drive/MyDrive/DRIVE_EXODUS_V2"
UNIT_ROOT = f"{DRIVE_ROOT}/01_ANIMATION_ENGINE"
CODEBASE = f"{UNIT_ROOT}/CODEBASE"

import sys
sys.path.insert(0, CODEBASE)

print("╔" + "═" * 50 + "╗")
print("║   TRANSMUTATION PRODUCTION — EXODUS SYSTEM        ║")
print("╚" + "═" * 50 + "╝")
print(f"\n✓ Environment configured")
print(f"✓ Drive Root: {DRIVE_ROOT}")

In [ ]:
# Install dependencies
!pip install -q torch torchvision opencv-python-headless scipy omegaconf Pillow tqdm

In [ ]:
# Setup Blender 4.0 (si pas déjà sur le Drive)
import os
from pathlib import Path

BLENDER_PATH = f"{DRIVE_ROOT}/EXODUS_AI_MODELS/blender-4.0.0-linux-x64/blender"

if not Path(BLENDER_PATH).exists():
    print("⚠ Blender 4.0 non trouvé sur le Drive")
    print("Téléchargement automatique...")
    
    !mkdir -p {DRIVE_ROOT}/EXODUS_AI_MODELS
    !wget -q https://download.blender.org/release/Blender4.0/blender-4.0.0-linux-x64.tar.xz -O /tmp/blender.tar.xz
    !tar -xf /tmp/blender.tar.xz -C {DRIVE_ROOT}/EXODUS_AI_MODELS/
    !rm /tmp/blender.tar.xz
    
    print("✓ Blender 4.0 installé")
else:
    print(f"✓ Blender 4.0 trouvé: {BLENDER_PATH}")

## 📋 Configuration Batch

In [ ]:
# Configuration des jobs batch
# Chaque job: (body_fbx, video, actor_blend, output_name, sync_offset)

BATCH_JOBS = [
    {
        "name": "ACTOR_01",
        "body_fbx": "dance_motion_01.fbx",
        "video": "face_capture_01.mp4",
        "actor_blend": "roblox_avatar_01.blend",
        "sync_offset": 0,
        "smooth_window": 5
    },
    {
        "name": "ACTOR_02",
        "body_fbx": "dance_motion_01.fbx",  # Même body motion
        "video": "face_capture_02.mp4",     # Visage différent
        "actor_blend": "roblox_avatar_02.blend",
        "sync_offset": 15,
        "smooth_window": 7
    },
    # Ajouter d'autres jobs ici...
]

print(f"Jobs configurés: {len(BATCH_JOBS)}")
for i, job in enumerate(BATCH_JOBS):
    print(f"  {i+1}. {job['name']}: {job['body_fbx']} + {job['video']} → {job['actor_blend']}")

## 🔍 Validation Pre-Flight

In [ ]:
# Vérifier que tous les fichiers existent avant de lancer
from pathlib import Path

def validate_job(job):
    """Valide qu'un job a tous ses fichiers."""
    errors = []
    
    body_path = Path(f"{UNIT_ROOT}/IN_INPUTS/body_motions/{job['body_fbx']}")
    if not body_path.exists():
        errors.append(f"Body FBX manquant: {job['body_fbx']}")
    
    video_path = Path(f"{UNIT_ROOT}/IN_INPUTS/source_videos/{job['video']}")
    if not video_path.exists():
        errors.append(f"Video manquante: {job['video']}")
    
    actor_path = Path(f"{UNIT_ROOT}/IN_INPUTS/actor_models/{job['actor_blend']}")
    if not actor_path.exists():
        errors.append(f"Actor manquant: {job['actor_blend']}")
    
    return errors

print("=== Validation Pre-Flight ===")
all_valid = True

for job in BATCH_JOBS:
    errors = validate_job(job)
    if errors:
        all_valid = False
        print(f"\n✗ {job['name']}:")
        for err in errors:
            print(f"   - {err}")
    else:
        print(f"✓ {job['name']}: tous les fichiers présents")

if all_valid:
    print("\n✓ VALIDATION OK — Prêt pour production")
else:
    print("\n✗ VALIDATION ÉCHOUÉE — Vérifiez les fichiers manquants")

## 🏭 Exécution Batch

In [ ]:
import subprocess
import time
from datetime import datetime

def run_transmutation(job):
    """Exécute le pipeline TRANSMUTATION pour un job."""
    start_time = time.time()
    
    cmd = [
        "python", f"{CODEBASE}/EXO_01_TRANSMUTATION.py",
        "--drive-root", DRIVE_ROOT,
        "--body-fbx", job["body_fbx"],
        "--video", job["video"],
        "--actor-blend", job["actor_blend"],
        "--output-name", job["name"],
        "--sync-offset", str(job.get("sync_offset", 0)),
        "--smooth-window", str(job.get("smooth_window", 5)),
        "-v"
    ]
    
    print(f"\n{'='*60}")
    print(f"PROCESSING: {job['name']}")
    print(f"Started: {datetime.now().strftime('%H:%M:%S')}")
    print(f"{'='*60}")
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    
    elapsed = time.time() - start_time
    
    if result.returncode == 0:
        print(f"\n✓ {job['name']} COMPLÉTÉ en {elapsed:.1f}s")
        return True, elapsed
    else:
        print(f"\n✗ {job['name']} ÉCHOUÉ")
        print(f"STDERR: {result.stderr}")
        return False, elapsed

In [ ]:
# Exécuter tous les jobs
results = []
total_start = time.time()

print("╔" + "═" * 50 + "╗")
print("║       TRANSMUTATION BATCH — DÉMARRAGE             ║")
print("╚" + "═" * 50 + "╝")
print(f"\nJobs à traiter: {len(BATCH_JOBS)}")

for i, job in enumerate(BATCH_JOBS):
    print(f"\n[{i+1}/{len(BATCH_JOBS)}] Processing {job['name']}...")
    
    # Validation
    errors = validate_job(job)
    if errors:
        print(f"  ⚠ Skipped (fichiers manquants)")
        results.append((job['name'], False, 0, "Fichiers manquants"))
        continue
    
    # Exécution
    success, elapsed = run_transmutation(job)
    results.append((job['name'], success, elapsed, "" if success else "Pipeline error"))

total_elapsed = time.time() - total_start

# Résumé
print("\n" + "=" * 60)
print("BATCH TERMINÉ — RÉSUMÉ")
print("=" * 60)

success_count = sum(1 for r in results if r[1])
print(f"\nSuccès: {success_count}/{len(results)}")
print(f"Temps total: {total_elapsed:.1f}s")

print("\nDétails:")
for name, success, elapsed, error in results:
    status = "✓" if success else "✗"
    print(f"  {status} {name}: {elapsed:.1f}s {error}")

## 📁 Vérification Outputs

In [ ]:
# Lister les fichiers générés
import os
from pathlib import Path

output_dir = Path(f"{UNIT_ROOT}/OUT_BAKED")

print("=== Fichiers Générés ===")
print(f"Dossier: {output_dir}\n")

if output_dir.exists():
    files = sorted(output_dir.glob("*"))
    
    abc_files = [f for f in files if f.suffix == '.abc']
    json_files = [f for f in files if f.suffix == '.json']
    
    print(f"Alembic (.abc): {len(abc_files)}")
    for f in abc_files:
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  📦 {f.name} ({size_mb:.2f} MB)")
    
    print(f"\nFace Data (.json): {len(json_files)}")
    for f in json_files:
        size_kb = f.stat().st_size / 1024
        print(f"  📄 {f.name} ({size_kb:.1f} KB)")
else:
    print("Dossier output non trouvé")

## 📊 Rapport de Production

In [ ]:
# Générer rapport de production
import json
from datetime import datetime

report = {
    "timestamp": datetime.now().isoformat(),
    "total_jobs": len(BATCH_JOBS),
    "successful": success_count,
    "failed": len(results) - success_count,
    "total_time_seconds": total_elapsed,
    "jobs": [
        {
            "name": name,
            "success": success,
            "time_seconds": elapsed,
            "error": error
        }
        for name, success, elapsed, error in results
    ]
}

report_path = f"{UNIT_ROOT}/OUT_BAKED/production_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
with open(report_path, 'w') as f:
    json.dump(report, f, indent=2)

print(f"✓ Rapport sauvegardé: {report_path}")
print(f"\n{json.dumps(report, indent=2)}")

---
## ✅ Production Complete

Les fichiers `.abc` sont prêts pour import dans:
- Blender (pour rendu)
- Unity (via Alembic Importer)
- Roblox (conversion nécessaire)

**Next Step**: Utiliser U02_LOGISTICS pour packager les assets.